In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
import datetime
from src import camap

## get lat/lon, upstream area of COMIDs  

In [5]:
ref = pd.read_csv("hrrid_comid_MCZ.csv").set_index("HRRID")
ref.head()

,COMID
HRRID,
38131,82000867
38132,82000868
35112,82000921
38133,82000869
1,82001507


In [32]:
channels_path = "~/yuta/ISRD/MCZ/src/src/hrr_binIO/channels.txt"
upstreams = pd.read_csv(channels_path, header=None, sep="\s+")
upstreams.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,38132,0,0,0,0,0,37.013520,37.013520,6.696478,2.678735e-04,0.035,38.023666,136.754860
1,2,38133,0,0,0,0,0,25.326638,25.326638,0.528920,1.000000e-09,0.035,32.918232,102.496211
2,3,19604,0,0,0,0,0,28.256127,28.256127,1.239506,8.844835e-04,0.035,34.316251,111.386993
3,4,19604,0,0,0,0,0,59.758866,59.758866,7.095190,1.123507e-04,0.035,45.615236,196.813399
4,5,38134,0,0,0,0,0,125.733554,125.733554,21.100961,1.416782e-05,0.035,60.515759,346.395132


In [33]:
upstreams_ = upstreams.iloc[:, [0, 8]]
upstreams_.columns = ["HRRID", "uarea"]
upstreams_ = upstreams_.assign(COMID=upstreams_.HRRID.apply(lambda x: ref.loc[x].values[0]))

In [34]:
path = "/project/uma_colin_gleason/yuta/SIRF/data/MCZ/geometry/Mackenzie_rivers.shp"
gdf = gpd.read_file(path)
gdf_ = gdf.loc[:, ["COMID", "geometry"]]
gdf.head()

,COMID,lengthkm,lengthdir,sinuosity,slope,uparea,order,strmDrop_t,slope_taud,NextDownID,maxup,up1,up2,up3,up4,LAKE,width_mean,width_max,geometry
0,82000867,22.749297,16.550855,1.374509,0.000009,1.790820e+06,8,0.2,0.000009,0,2,82000868,82000921,0,0,0,4934.750000,6564.356934,"LINESTRING (-136.1041666666667 68.89, -136.103..."
1,82000868,12.321202,9.195698,1.339888,0.000032,1.781736e+06,8,0.4,0.000032,82000867,2,82000869,82001507,0,0,0,1290.969971,5085.518066,"LINESTRING (-135.7225 68.83333333333334, -135...."
2,82000921,0.527921,0.457208,1.154662,0.000000,8.906420e+03,5,0.0,0.000000,82000867,2,82000922,82001282,0,0,0,5276.750000,5297.905273,"LINESTRING (-135.7225 68.83333333333334, -135...."
3,82000869,4.375071,3.611140,1.211548,0.000229,1.781640e+06,8,0.0,0.000000,82000868,2,82000870,82001484,0,0,0,608.166992,863.368774,LINESTRING (-135.5533333333333 68.889166666666...
4,82001507,6.696478,4.377739,1.529666,0.000268,3.701352e+01,1,1.8,0.000268,82000868,0,0,0,0,0,0,210.785004,451.785828,LINESTRING (-135.5533333333333 68.889166666666...


In [35]:
gdf_ = gdf_.assign(lat=gdf_.geometry.apply(lambda x: x.coords[-1][1]))
gdf_ = gdf_.assign(lon=gdf_.geometry.apply(lambda x: x.coords[-1][0]))
gdf_ = gdf_.set_index("COMID")

In [36]:
mcz_props = upstreams_
mcz_props = mcz_props.assign(lat=mcz_props.COMID.apply(lambda x: gdf_.loc[x, "lat"]))
mcz_props = mcz_props.assign(lon=mcz_props.COMID.apply(lambda x: gdf_.loc[x, "lon"]))
mcz_props.head()

,HRRID,uarea,COMID,lat,lon
0,1,37.013520,82001507,68.919167,-135.624167
1,2,25.326638,82001484,68.901667,-135.465000
2,3,28.256127,82001374,68.811667,-135.341667
3,4,59.758866,82001419,68.851667,-135.335833
4,5,125.733554,82001586,69.003333,-135.368333


In [37]:
mcz_props.iloc[:, 1::].to_csv("./MCZ_COMID_location.csv", index=False)

In [38]:
cama = camap.Camap()

This is a toolkit to geomap dataset onto CaMa-Flood maps.
Register your json file first via .register(configjson).


In [39]:
cama.register("./src/camainfo_mackenzie.json")

In [40]:
lons = mcz_props.lon.values
lats = mcz_props.lat.values
upareas = mcz_props.uarea.values * 1000000

In [41]:
glons, glats, errors = cama.mapgrid_uparea(lons, lats, upareas, buffer=10)

In [42]:
glons = np.array(glons)
glats = np.array(glats)
errors = np.array(errors)
glons = np.where(errors > 0.1, -9999, glons)
glats = np.where(errors > 0.1, -9999, glats)

In [43]:
mcz_props = mcz_props.assign(glat=glats.astype(np.int32))
mcz_props = mcz_props.assign(glon=glons.astype(np.int32))
mcz_props.loc[mcz_props.glat==-9999, :]

,HRRID,uarea,COMID,lat,lon,glat,glon
20189,20190,332.800778,82008046,65.470000,-123.075000,-9999,-9999
20287,20288,173.623140,82012547,64.128333,-129.450833,-9999,-9999
24341,24342,343.401096,82007029,65.976667,-125.591667,-9999,-9999
24458,24459,280.389062,82012482,63.855000,-128.793333,-9999,-9999
26893,26894,572.772534,82016233,62.808333,-125.903333,-9999,-9999
26976,26977,399.895726,82007791,65.743333,-123.880000,-9999,-9999
27120,27121,541.553897,82028442,58.792500,-124.149167,-9999,-9999
27164,27165,475.507617,82019734,61.585833,-113.638333,-9999,-9999
27255,27256,555.382710,82030531,58.649167,-124.336667,-9999,-9999
27261,27262,280.609081,82025450,59.875833,-118.050000,-9999,-9999


In [44]:
mcz_props.iloc[:, 1::].to_csv("./MCZ_COMID_location.csv", index=False)

## save gauge data to netCDF  

In [2]:
gauge = pd.read_csv("./gauge_cal.txt", sep="\s+")
gauge.head()

,92,1441,1725,1894,5754,6449,6896,7960,14783,15283,...,38852,38893,38969,39081,39092,39126,39151,39161,39163,39175
0,-999.0,-999.0,0.155,-999.0,-999.000,-999.0,0.170,0.064,-999.0,-999.0,...,1800,-999,-999,1640,113.0,-999,-999,1650,1380,1380
1,-999.0,-999.0,0.152,-999.0,0.012,-999.0,0.170,0.064,-999.0,-999.0,...,1810,-999,-999,1650,112.0,-999,-999,1640,1400,1400
2,-999.0,-999.0,0.149,-999.0,0.012,-999.0,0.165,0.063,-999.0,-999.0,...,1820,-999,-999,1710,111.0,-999,-999,1610,1420,1290
3,-999.0,-999.0,0.147,-999.0,0.014,-999.0,0.160,0.063,-999.0,-999.0,...,1830,-999,-999,1760,111.0,-999,-999,1300,1280,1190
4,-999.0,-999.0,0.144,-999.0,0.014,-999.0,0.160,0.062,-999.0,-999.0,...,1830,-999,-999,1620,113.0,-999,-999,1100,1000,838


In [3]:
sdate = datetime.datetime(1984,1,1)
dates = pd.date_range(sdate, periods=len(gauge), freq="D")
gauge = gauge.replace(-999, np.nan)
array = np.expand_dims(gauge.values, axis=0)
dates

DatetimeIndex(['1984-01-01', '1984-01-02', '1984-01-03', '1984-01-04',
               '1984-01-05', '1984-01-06', '1984-01-07', '1984-01-08',
               '1984-01-09', '1984-01-10',
               ...
               '2013-12-22', '2013-12-23', '2013-12-24', '2013-12-25',
               '2013-12-26', '2013-12-27', '2013-12-28', '2013-12-29',
               '2013-12-30', '2013-12-31'],
              dtype='datetime64[ns]', length=10958, freq='D')

In [6]:
gauge.columns = pd.Series(gauge.columns.astype(int)).apply(lambda x: ref.loc[x].values[0])
gauge.head()

,82002220,82006869,82009553,82009496,82023032,82017747,82017892,82017269,82033201,82039396,...,82028201,82029837,82029872,82036781,82040772,82037779,82037800,82037816,82037818,82037835
0,NaN,NaN,0.155,NaN,NaN,NaN,0.170,0.064,NaN,NaN,...,1800.0,NaN,NaN,1640,113.0,NaN,NaN,1650.0,1380.0,1380
1,NaN,NaN,0.152,NaN,0.012,NaN,0.170,0.064,NaN,NaN,...,1810.0,NaN,NaN,1650,112.0,NaN,NaN,1640.0,1400.0,1400
2,NaN,NaN,0.149,NaN,0.012,NaN,0.165,0.063,NaN,NaN,...,1820.0,NaN,NaN,1710,111.0,NaN,NaN,1610.0,1420.0,1290
3,NaN,NaN,0.147,NaN,0.014,NaN,0.160,0.063,NaN,NaN,...,1830.0,NaN,NaN,1760,111.0,NaN,NaN,1300.0,1280.0,1190
4,NaN,NaN,0.144,NaN,0.014,NaN,0.160,0.062,NaN,NaN,...,1830.0,NaN,NaN,1620,113.0,NaN,NaN,1100.0,1000.0,838


In [7]:
darray = xr.DataArray(array, dims=["kind", "time", "reach"], coords=[["discharge"], dates, gauge.columns])
darray.name = "gauge"
darray

<xarray.DataArray 'gauge' (kind: 1, time: 10958, reach: 327)>
array([[[  nan,   nan, ..., 1380., 1380.],
        [  nan,   nan, ..., 1400., 1400.],
        ...,
        [  nan,   nan, ...,   nan, 1280.],
        [  nan,   nan, ...,   nan, 1270.]]])
Coordinates:
  * kind     (kind) <U9 'discharge'
  * time     (time) datetime64[ns] 1984-01-01 1984-01-02 ... 2013-12-31
  * reach    (reach) int64 82002220 82006869 82009553 ... 82037818 82037835

In [8]:
dset = darray.to_dataset()
dset.to_netcdf("gauge.nc")

In [49]:
gauge_props = mcz_props.set_index("COMID").loc[gauge.columns, :].reset_index()[["index", "lat", "lon", "uarea", "glat", "glon"]]
gauge_props.columns = ["COMID", "lat", "lon", "uparea", "glat", "glon"]
gauge_props.to_csv("gauge_location_MCZ.csv", index=False)

## save geobam data to netCDF  

In [14]:
src = "./geoBAMr_unsupervised_vic_hrr.csv"
df = pd.read_csv(src, parse_dates=[0])
df.head()

,Date,reach,Mean,Std
0,1984-06-09,26061,0.893405,1.373807
1,1984-08-03,26061,0.891601,1.384467
2,1984-08-12,26061,0.907083,1.572716
3,1984-08-21,26061,1.006573,1.410237
4,1985-05-20,26061,0.975695,1.366836


In [15]:
mean = df.pivot(index="Date", columns="reach", values="Mean")
std = df.pivot(index="Date", columns="reach", values="Std")
reaches = mean.columns.astype(np.int32)

In [16]:
comids = pd.Series(reaches).apply(lambda x: ref.loc[x].values[0])

In [17]:
dates = mean.index
mean_array = np.expand_dims(mean.values, axis=0)
std_array = np.expand_dims(std.values, axis=0)
array = np.vstack([mean_array, std_array])
darray = xr.DataArray(array,
                      dims=["kind", "time", "reach"], 
                      coords=[["mean", "std"], dates, reaches])

In [18]:
darray

<xarray.DataArray (kind: 2, time: 3981, reach: 7861)>
array([[[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]]])
Coordinates:
  * kind     (kind) <U4 'mean' 'std'
  * time     (time) datetime64[ns] 1984-04-07 1984-04-09 ... 2013-10-30
  * reach    (reach) int64 1 4 5 6 7 8 ... 39168 39169 39170 39171 39174 39175

In [19]:
darray.name = "geobam"
dset = darray.to_dataset()
dset.to_netcdf("geobam.nc")

## take mean of width in each 

In [20]:
width = pd.read_parquet("./width_MCZ.parquet")

In [21]:
width.Date = pd.to_datetime(width.Date)
width = width.replace(0, np.nan)
width.head(20)

,reach,xs,Date,width
0,82027041,209217,1984-08-21 17:26:24,91.927428
1,82027041,209218,1984-08-21 17:26:24,203.131660
2,82027041,209219,1984-08-21 17:26:24,265.518220
3,82027041,209220,1984-08-21 17:26:24,236.936283
4,82027041,209221,1984-08-21 17:26:24,296.052753
5,82027041,209222,1984-08-21 17:26:24,247.860447
6,82027041,209223,1984-08-21 17:26:24,151.517844
7,82027041,209224,1984-08-21 17:26:24,2461.237928
8,82027041,209225,1984-08-21 17:26:24,1331.619783
9,82027041,209226,1984-08-21 17:26:24,NaN


In [22]:
width.Date = pd.Series(width.set_index("Date").index.floor("D"))

In [23]:
width.head()

,reach,xs,Date,width
0,82027041,209217,1984-08-21,91.927428
1,82027041,209218,1984-08-21,203.131660
2,82027041,209219,1984-08-21,265.518220
3,82027041,209220,1984-08-21,236.936283
4,82027041,209221,1984-08-21,296.052753


In [24]:
width_grouped = width.groupby(["Date", "reach"]).mean()

In [25]:
width_wide = width_grouped.reset_index().pivot(index="Date", columns="reach", values="width")

In [26]:
width_wide.head(20)

reach,82000425,82000573,82000867,82000868,82000869,82000870,82000871,82000872,82000873,82000874,...,82043055,82043057,82043058,82043059,82043060,82043061,82043062,82043065,82043073,82043076
Date,,,,,,,,,,,,,,,,,,,,,
1984-04-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984-04-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984-04-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984-04-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984-04-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
width_array = width_wide.values
width_array.shape

(5018, 9234)

In [28]:
comids = width_wide.columns.tolist()
dates = width_wide.index.tolist()

In [29]:
darray = xr.DataArray(np.expand_dims(width_array, axis=0), dims=["kind", "time", "reach"], coords=[["width"], dates, comids])
darray.name = "width"
darray

<xarray.DataArray 'width' (kind: 1, time: 5018, reach: 9234)>
array([[[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]]])
Coordinates:
  * kind     (kind) <U5 'width'
  * time     (time) datetime64[ns] 1984-04-07 1984-04-09 ... 2019-08-14
  * reach    (reach) int64 82000425 82000573 82000867 ... 82043073 82043076

In [30]:
darray.to_netcdf("./width_MCZ.nc")